### Load dataset

In [72]:
import pandas as pd
RANDOM_STATE = 42

%run "DataHelpers.ipynb"

# Can be replaced with desired variant for different feature sets
GENE_FILE_VARIANT = FeatureVariant.RESEARCH_LOGSCALED # For values, see FeatureVariant.print_info()
variant = ModelVariant.SVM                      # For values, see ModelVariant.print_info()

FILE_PATH = f"../Data/patient_genes_{GENE_FILE_VARIANT}.csv"

df = pd.read_csv(FILE_PATH)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 33 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   FYN      977 non-null    float64
 1   BRCA1    977 non-null    float64
 2   FOXC1    977 non-null    float64
 3   TBC1D1   977 non-null    float64
 4   LAG3     977 non-null    float64
 5   CDK6     977 non-null    float64
 6   GATA3    977 non-null    float64
 7   CCND1    977 non-null    float64
 8   PRR4     977 non-null    float64
 9   EPCAM    977 non-null    float64
 10  CD274    977 non-null    float64
 11  PIK3CA   977 non-null    float64
 12  TOP2A    977 non-null    float64
 13  DCLK1    977 non-null    float64
 14  MYC      977 non-null    float64
 15  LRPPRC   977 non-null    float64
 16  BRCA2    977 non-null    float64
 17  TP53     977 non-null    float64
 18  EGFR     977 non-null    float64
 19  MKI67    977 non-null    float64
 20  TTN      977 non-null    float64
 21  CTLA4    977 non

### Import model function

In [73]:
from sklearn.svm import SVC

### Dataset split: training and test data

In [74]:
X, y, X_train, X_test, y_train, y_test, test_case_ids = split_data(df, "tnbc", True)
print("\nApplied Smote")
X_smote, y_smote, X_train_smote, X_test_smote, y_train_smote, y_test_smote, test_case_ids_smote = split_data_apply_smote(df, "tnbc")

X_train.shape=(781, 31)
X_test.shape=(196, 31)
y_train.shape=(781,)
y_test.shape=(196,)

Applied Smote
X_train.shape=(1379, 31)
X_test.shape=(345, 31)
y_train.shape=(1379,)
y_test.shape=(345,)


### Support Vector Machine (SVM)

In [75]:
# Create model

model = SVC(random_state=RANDOM_STATE, probability=True)

def run_model(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series, test_case_ids: pd.Series, is_smote: bool):
    # Train the model
    model.fit(X_train, y_train)

    # Model predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # For ROC curves etc.

    # Save it in a dataframe, to CSV
    predictions = pd.DataFrame({
        "case_id": test_case_ids,
        "y_test": y_test,
        "y_pred": y_pred,
        "y_prob": y_prob
    })
    predictions.to_csv(f"../Data/model_output_{variant}_{GENE_FILE_VARIANT}{'_smote' if is_smote else ''}.csv", index=False)

    return y_pred, y_prob

In [76]:
y_pred, y_prod = run_model(X_train, X_test, y_train, y_test, test_case_ids, False)
print_evaluated_model_accuracy(y_test, y_pred)

Accuracy: 0.94


## Smote applied

In [77]:
y_pred_smote, y_prod_smote = run_model(X_train_smote, X_test_smote, y_train_smote, y_test_smote, test_case_ids_smote, True)

print_evaluated_model_accuracy(y_test_smote, y_pred_smote)

Accuracy: 0.98


## Model cross validation

In [78]:
def run_cross_validation(X: pd.DataFrame, y: pd.Series, y_test: pd.Series, y_pred: pd.Series, y_prob: pd.Series, is_smote: bool) -> pd.DataFrame:
    metrics: pd.DataFrame = get_cross_validation_metrics(model, X, y, cv=5)
    test_metrics = get_metrics(y_test, y_pred, y_prob)
    test_metrics["fold"] = 0 # Initial test metrics (before cross validation)
    test = pd.DataFrame([test_metrics])
    test.set_index("fold", inplace=True)

    print_validated_model_accuracy(model, metrics)

    # Prepend test_metrics to metrics dataframe, export and display
    metrics = pd.concat([test, metrics])
    metrics.to_csv(f"../Data/model_metrics_{variant}_{GENE_FILE_VARIANT}{'_smote' if is_smote else ''}.csv", index=False)
    return metrics

In [79]:
metrics = run_cross_validation(X, y, y_test, y_pred, y_prod, False)
metrics

Model validation for SVC:
[0.9438775510204082, 0.9489795918367347, 0.9384615384615385, 0.9333333333333333, 0.9333333333333333]

Mean accuracy: 0.9396



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.943878,0.782609,0.750000,0.765957,0.981654,18,167,6,5
1,0.943878,0.782609,0.750000,0.765957,0.966323,18,167,6,5
2,0.948980,0.782609,0.782609,0.782609,0.951244,18,168,5,5
3,0.938462,0.652174,0.789474,0.714286,0.964863,15,168,4,8
4,0.933333,0.652174,0.750000,0.697674,0.971436,15,167,5,8
5,0.933333,0.695652,0.727273,0.711111,0.926946,16,166,6,7


In [80]:
metric_smote = run_cross_validation(X_smote, y_smote, y_test_smote, y_pred_smote, y_prod_smote, True)
metric_smote

Model validation for SVC:
[0.9652173913043478, 0.9710144927536232, 0.9739130434782609, 0.9623188405797102, 0.9680232558139535]

Mean accuracy: 0.9681



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.976812,1.000000,0.955556,0.977273,0.993984,172,165,8,0
1,0.965217,0.994186,0.939560,0.966102,0.993816,171,162,11,1
2,0.971014,0.994186,0.950000,0.971591,0.987028,171,164,9,1
3,0.973913,0.976879,0.971264,0.974063,0.991531,169,167,5,4
4,0.962319,0.982659,0.944444,0.963173,0.987935,170,162,10,3
5,0.968023,0.994186,0.944751,0.968839,0.991144,171,162,10,1
